In [21]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3.1:8b', reasoning=False, validate_model_on_init=True, temperature=0)

In [34]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "weather": {
            "transport": "stdio",
            "command": "python",
            "args": ["./weather.py"],
        },
        "calculator": {
            "transport": "stdio",
            "command": "python",
            "args": ["./calculator.py"],
        }
    }
)

tools = await client.get_tools()
print(len(tools))
for t in tools:
    print(f'Name: {t.name}, Description: {t.description}')

5
Name: get_forecast_us, Description: Get weather alerts for a location in US. Input in longitude and latitude
Name: get_forecast_can, Description: Get weather alerts for location in Canada. Input in longitude and latitude
Name: add_numbers, Description: Add numbers together and return the sum.
Name: subtract, Description: Subtract the second number from the first and return the difference.
Name: multiply_numbers, Description: Multiply numbers together and return the product.


In [23]:
from pydantic import BaseModel, Field

class WeatherData(BaseModel):
    temperature: str = Field(..., description="Temperature without units, e.g. 20")
    unit: str = Field(..., description="Temperature unit, e.g. °C or °F")

In [24]:
from langchain.agents.structured_output import ToolStrategy
from langchain.agents import create_agent

agent = create_agent(model=llm, tools=tools)

In [25]:
from langchain.messages import HumanMessage, SystemMessage

In [26]:
async def invoke_agent_with_logging(message: HumanMessage):
    messages = [SystemMessage(content="You are a helpful assistant."), message]
    print("\nAgent Response:")
    response = await agent.ainvoke({ 'messages': messages })
    for msg in response['messages']:
        if msg.type == 'ai':
            for tool_call in msg.tool_calls:
                print(f"Tool Call - Name: {tool_call['name']}, Args: {tool_call['args']}")
        

    print(response['messages'][-1].content)

In [27]:
msg = HumanMessage(content="What is the weather in Buffalo, NY tomorrow morning?")
await invoke_agent_with_logging(msg)


Agent Response:
Tool Call - Name: get_forecast_us, Args: {'latitude': 42.8863, 'longitude': -78.8927}
Based on the forecast, it will be rainy in Buffalo, NY tomorrow morning. The temperature is expected to be around 52°F (11°C) with a low chance of rain showers after 4am.


In [28]:
msg = HumanMessage(content="What is the weather in Ajax, ON tomorrow morning?")
await invoke_agent_with_logging(msg)


Agent Response:
Tool Call - Name: get_forecast_can, Args: {'latitude': 43.85, 'longitude': -79.02}
Based on the forecast, it will be cloudy with a high temperature of 23°C and a low of 10°C tomorrow morning in Ajax, ON. There is also a risk of thunderstorms late this afternoon.


In [29]:
msg = HumanMessage(content="What is the weather in Ajax, ON and Buffalo tomorrow morning?")
await invoke_agent_with_logging(msg)


Agent Response:
Tool Call - Name: get_forecast_can, Args: {'latitude': 43.8597, 'longitude': -79.4872}
Tool Call - Name: get_forecast_us, Args: {'latitude': 42.8863, 'longitude': -78.8879}
The weather in Ajax, ON tomorrow morning will be cloudy with a high of 22°C and a low of 10°C. There is a risk of showers beginning this afternoon.

The weather in Buffalo, NY tomorrow morning will be mostly cloudy with a high of 75°F (23.9°C) and a low of 52°F (11.1°C). There is a chance of rain showers before noon and again in the evening.


In [30]:
msg = HumanMessage(content="What is the sum of 12345 and 67890?")
await invoke_agent_with_logging(msg)


Agent Response:
Tool Call - Name: add_numbers, Args: {'values': [12345, 67890]}
The sum of 12345 and 67890 is 80235.


In [31]:
prompt = await client.get_prompt('weather', 'weather_us', arguments={'latitude': '-43', 'longitude': '-71'})
await invoke_agent_with_logging(prompt[0])


Agent Response:
Tool Call - Name: get_forecast_us, Args: {'latitude': -43, 'longitude': -71}
The current temperature in the specified location is 12°C.
